## Set up

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import requests

## Get NYT county COVID data

In [2]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"

# Save to a dataframe
covid_all_df = pd.read_csv(counties_path)

#### Build all values dataframe

In [3]:
# Drop rows with null values (we want complete data only)
covid_all_df = covid_all_df.dropna()

# Convert fips to int
covid_all_df = covid_all_df.astype({'fips': 'int'})

In [4]:
# Display the dataframe
covid_all_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
579818,2020-09-28,Sweetwater,Wyoming,56037,334,2
579819,2020-09-28,Teton,Wyoming,56039,560,1
579820,2020-09-28,Uinta,Wyoming,56041,354,2
579821,2020-09-28,Washakie,Wyoming,56043,114,6


#### Build latest values dataframe

In [5]:
# Get the most up-to-date data
latest_date = covid_all_df['date'].iloc[-1]

# Make a dataframe holding that most recent data
covid_latest_df = covid_all_df.loc[covid_all_df['date']==latest_date]

In [6]:
# Display the dataframe
covid_latest_df

,date,county,state,fips,cases,deaths
576582,2020-09-28,Autauga,Alabama,1001,1785,25
576583,2020-09-28,Baldwin,Alabama,1003,5588,50
576584,2020-09-28,Barbour,Alabama,1005,886,7
576585,2020-09-28,Bibb,Alabama,1007,657,10
576586,2020-09-28,Blount,Alabama,1009,1618,15
...,...,...,...,...,...,...
579818,2020-09-28,Sweetwater,Wyoming,56037,334,2
579819,2020-09-28,Teton,Wyoming,56039,560,1
579820,2020-09-28,Uinta,Wyoming,56041,354,2
579821,2020-09-28,Washakie,Wyoming,56043,114,6


## Get mask use data

In [7]:
# Import the mask CSV data
mask_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv"
masks_df = pd.read_csv(mask_path)

In [8]:
# Clean up column names
masks_df.columns = ['fips','never','rarely','sometimes','frequently','always']

In [9]:
# Display the dataframe
masks_df

,fips,never,rarely,sometimes,frequently,always
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


## Get CDC data
#### Source: https://data.cdc.gov/NCHS/Weekly-Counts-of-Deaths-by-State-and-Select-Causes/muzy-jte6

In [10]:
cdc_url = 'https://data.cdc.gov/resource/muzy-jte6.json'
response = requests.get(cdc_url)
cdc_json = response.json()

# Convert to a dataframe
cdc_df = pd.DataFrame.from_dict(cdc_json)

In [11]:
# Rename columns
cdc_df.columns = ['state',
                  'year',
                  'week',
                  'week_ending_date',
                  'all_causes',
                  'natural_causes',
                  'septicemia',
                  'malignant_neoplasms',
                  'diabetes',
                  'alzheimers',
                  'influenza_and_pneumonia',
                  'chronic_lower_respiratory',
                  'other_diseases_of_respiratory',
                  'nephritis_nephrotic_syndrome',
                  'symptoms_signs_and_abnormal',
                  'diseases_of_heart',
                  'cerebrovascular_diseases',
                  'covid_19_multiple_causes',
                  'covid_19_underlying_cause',
                  'flag_otherresp',
                  'flag_otherunk',
                  'flag_nephr',
                  'flag_inflpn',
                  'flag_cov19mcod',
                  'flag_cov19ucod',
                  'flag_sept',
                  'flag_diab',
                  'flag_alz',
                  'flag_clrd',
                  'flag_stroke',
                  'flag_hd',
                  'flag_neopl',
                  'flag_allcause',
                  'flag_natcause']     

In [12]:
# Display the dataframe
cdc_df

,state,year,week,week_ending_date,all_causes,natural_causes,septicemia,malignant_neoplasms,diabetes,alzheimers,...,flag_cov19ucod,flag_sept,flag_diab,flag_alz,flag_clrd,flag_stroke,flag_hd,flag_neopl,flag_allcause,flag_natcause
0,Alabama,2019,1,2019-01-05,1077,993,30,198,22,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2019,2,2019-01-12,1090,994,25,187,24,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2019,3,2019-01-19,1114,1042,22,238,18,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,2019,4,2019-01-26,1063,994,21,165,22,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2019,5,2019-02-02,1095,1026,18,199,19,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Hawaii,2019,17,2019-04-27,221,211,NaN,49,NaN,NaN,...,NaN,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN
996,Hawaii,2019,18,2019-05-04,209,190,NaN,37,NaN,NaN,...,NaN,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN
997,Hawaii,2019,19,2019-05-11,246,232,NaN,52,NaN,NaN,...,NaN,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN
998,Hawaii,2019,20,2019-05-18,233,213,NaN,53,NaN,NaN,...,NaN,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),NaN,NaN,NaN,NaN,NaN


In [13]:
# Build 2019 and 2020 dataframes, just in case
cdc_2019_df = cdc_df.loc[cdc_df['year']=="2019"]
cdc_2020_df = cdc_df.loc[cdc_df['year']=="2020"]

## Get Covid Tracking Project data
Source: https://covidtracking.com

#### Build latest values dataframe

In [14]:
covidtracking_current_url = 'https://api.covidtracking.com/v1/states/current.json'
covidtracking_current_response = requests.get(covidtracking_current_url)
covidtracking_current_json = covidtracking_current_response.json()

# Convert to a dataframe
covidtracking_current_df = pd.DataFrame.from_dict(covidtracking_current_json)

In [15]:
# Display the dataframe
covidtracking_current_df

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200928,AK,8549,436242,NaN,444791,43.0,NaN,NaN,NaN,...,444791,0,0,4e69422096acf75151468261082a237436cef891,0,0,0,0,0,
1,20200928,AL,152983,980291,NaN,1116346,753.0,17091.0,NaN,1798.0,...,1133274,0,239,7c8d97e72cc9e00f4d59d877297d6413d2f49138,0,0,0,0,0,
2,20200928,AR,82049,918177,NaN,997698,491.0,5248.0,231.0,NaN,...,1000226,21,13,b709c3e13433dcdeb90a274d7e3353f0eb886bf9,0,0,0,0,0,
3,20200928,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,...,1571,0,0,d3ac2fc4e017ac48460c41a526bea4251d80cc7f,0,0,0,0,0,
4,20200928,AZ,217510,1235742,NaN,1448712,468.0,22019.0,115.0,NaN,...,1453252,0,4,76daa017cbe7b18563b3d021f9b623f1d3a30ec9,0,0,0,0,0,
5,20200928,CA,805263,13679589,NaN,14484852,3160.0,NaN,831.0,NaN,...,14484852,21,0,1e3dec497984a20226f60fd6a4bef28d2c950802,0,0,0,0,0,
6,20200928,CO,69079,824302,NaN,1313403,263.0,7523.0,NaN,NaN,...,893381,0,3,1c103433ab72a02dfc873988a610a50ebfaa5fa4,0,0,0,0,0,
7,20200928,CT,57147,1514059,NaN,1571206,75.0,11560.0,NaN,NaN,...,1571206,2,0,cce8c1b9eddcbda868fbe5f1ca60af442efeb5a7,0,0,0,0,0,
8,20200928,DC,15264,367784,NaN,383048,90.0,NaN,27.0,NaN,...,383048,0,0,facd4d9a54810759d5d18388c5ff6e54b99f6fbb,0,0,0,0,0,
9,20200928,DE,20389,263426,NaN,283815,60.0,NaN,13.0,NaN,...,283815,1,0,a6b318695f876cf27eaf85294c9c6acdac57ebdb,0,0,0,0,0,


#### Build latest values dataframe

In [16]:
covidtracking_all_url = 'https://api.covidtracking.com/v1/us/daily.json'
covidtracking_all_response = requests.get(covidtracking_all_url)
covidtracking_all_json = covidtracking_all_response.json()

# Convert to a dataframe
covidtracking_all_df = pd.DataFrame.from_dict(covidtracking_all_json)

In [17]:
# Display the dataframe
covidtracking_all_df

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20200928,56,7117251,91574039,11160.0,29539.0,405326.0,5974.0,20121.0,1518.0,...,102342416,2020-09-28T00:00:00Z,0,0,257,1241,919021,36524,1010138,981ef3a0f4d5aaad26d07479fa531d90778a7331
1,20200927,56,7080727,90655018,11136.0,29434.0,404085.0,6080.0,20050.0,1511.0,...,101332278,2020-09-27T00:00:00Z,0,0,307,760,669524,35454,825248,19b8b285032c1ec14392f20904a8d0ac3d57dac2
2,20200926,56,7045273,89985494,11183.0,29554.0,403325.0,6057.0,20002.0,1509.0,...,100507030,2020-09-26T00:00:00Z,0,0,869,1154,889151,47836,1018755,16075f11d704d0c5befc819ae4a9a32f47069729
3,20200925,56,6997437,89096343,10905.0,29769.0,402171.0,6133.0,19919.0,1506.0,...,99488275,2020-09-25T00:00:00Z,0,0,844,1331,856519,55526,1011675,8d311e73fe038522a1a6be4bc3202de206ec0adb
4,20200924,56,6941911,88239824,12008.0,30043.0,400840.0,6168.0,19555.0,1560.0,...,98476600,2020-09-24T00:00:00Z,0,0,921,1588,823449,43772,940353,375a88dd29991abc1946cd7f98f4f20a9e37fb5d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,20200126,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-26T00:00:00Z,0,0,0,0,0,0,0,88a196879207bd090a61fb515beb301e84377977
247,20200125,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-25T00:00:00Z,0,0,0,0,0,0,0,43330e2f11d476ebcdd85e1a47551958e4c1e7ea
248,20200124,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-24T00:00:00Z,0,0,0,0,0,0,0,5f7a0cce04041596513629d55777a16f3aa65e8b
249,20200123,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-23T00:00:00Z,0,0,0,0,0,0,1,53f6a5e3985cebdead7ca275b0890ef975f0ccdd


## Get the county information table

In [18]:
# Scrape the county info table from Wikipedia
county_url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
county_table = pd.read_html(county_url)

In [19]:
# Grab the first table on the page and convert to dataframe
county_table_df = county_table[0]

# Drop columns we don't need
county_table_df = county_table_df.drop(columns=['Land Areakm²','Land Areami²','Water Areakm²','Water Areami²','Total Areakm²','Total Areami²','Sort [1]','Population(2010)'])

# Rename the columns
county_table_df.columns = ['state','fips','county','county_seat','lat','lon']

# Remove the + sign from latitude column
county_table_df['lat'] = county_table_df['lat'].str[1:]
county_table_df['lon'] = county_table_df['lon'].str[1:]

# Remove degree symbol from lat and lon
county_table_df['lat'] = county_table_df['lat'].str[:-1]
county_table_df['lon'] = county_table_df['lon'].str[:-1]

# Set as a float for lat and lon
county_table_df['lat'] = county_table_df['lat'].astype(float)
county_table_df['lon'] = county_table_df['lon'].astype(float)

In [20]:
# Convert the lon to an actual negative value (for all, since North America)
county_table_df['lon'] = -county_table_df['lon']

In [21]:
# Display the dataframe
county_table_df

,state,fips,county,county_seat,lat,lon
0,AL,1001,Autauga,Prattville,32.536382,-86.644490
1,AL,1003,Baldwin,Bay Minette,30.659218,-87.746067
2,AL,1005,Barbour,Clayton,31.870670,-85.405456
3,AL,1007,Bibb,Centreville,33.015893,-87.127148
4,AL,1009,Blount,Oneonta,33.977448,-86.567246
...,...,...,...,...,...,...
3138,WY,56037,Sweetwater,Green River,41.660339,-108.875676
3139,WY,56039,Teton,Jackson,44.049321,-110.588102
3140,WY,56041,Uinta,Evanston,41.284726,-110.558947
3141,WY,56043,Washakie,Worland,43.878831,-107.669052


## Connect and load to postgressql
### Available dataframes:
<b>NYT:</b> covid_all_df, covid_latest_df <br>
<b>Masks:</b> masks_df <br>
<b>CDC:</b> cdc_df, cdc_2019_df, cdc_2020_df <br>
<b>Covid tracking:</b> covidtracking_current_df, covidtracking_all_df<br>
<b>County info:</b> county_table_df

In [22]:
# Connect to local database
# Make sure you fill out the user / password for MySQL
rds_connection_string = "covid_db_admin:pass123@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [23]:
# Check for Tables
engine.table_names()

['cdc', 'covid', 'masks', 'county']

### Load data into database

In [24]:
# Load covid_all_df dataframe into database
covid_all_df.to_sql(name='covid', con=engine, if_exists='append', index=False)

In [25]:
# Load mask_df dataframe into database
masks_df.to_sql(name='masks', con=engine, if_exists='append', index=False)

In [26]:
# Load cdc_df dataframe into database
cdc_df.to_sql(name='cdc', con=engine, if_exists='append', index=False)

In [27]:
# Not yet working
#covidtracking_current_df.to_sql(name='covidtracking_current', con=engine, if_exists='append', index=False)

In [28]:
# Not yet working
#covidtracking_all_df.to_sql(name='covidtracking_all', con=engine, if_exists='append', index=False)

In [29]:
# Load county_table_df dataframe into database
county_table_df.to_sql(name='county', con=engine, if_exists='append', index=False)

### Confirm data load

In [30]:
# Confirm covid data has been added
pd.read_sql_query('select * from covid', con=engine)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0
...,...,...,...,...,...,...
574233,2020-09-28,Sweetwater,Wyoming,56037,334,2
574234,2020-09-28,Teton,Wyoming,56039,560,1
574235,2020-09-28,Uinta,Wyoming,56041,354,2
574236,2020-09-28,Washakie,Wyoming,56043,114,6


In [31]:
# Confirm mask use data has been added
pd.read_sql_query('select * from masks', con=engine)

,fips,never,rarely,sometimes,frequently,always
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


In [32]:
# Confirm cdc data has been added
pd.read_sql_query('select * from cdc', con=engine)

,state,year,week,week_ending_date,all_causes,natural_causes,septicemia,malignant_neoplasms,diabetes,alzheimers,...,flag_cov19ucod,flag_sept,flag_diab,flag_alz,flag_clrd,flag_stroke,flag_hd,flag_neopl,flag_allcause,flag_natcause
0,Alabama,2019,1,2019-01-05,1077.0,993.0,30.0,198.0,22.0,60.0,...,None,None,None,None,None,None,None,None,None,None
1,Alabama,2019,2,2019-01-12,1090.0,994.0,25.0,187.0,24.0,49.0,...,None,None,None,None,None,None,None,None,None,None
2,Alabama,2019,3,2019-01-19,1114.0,1042.0,22.0,238.0,18.0,48.0,...,None,None,None,None,None,None,None,None,None,None
3,Alabama,2019,4,2019-01-26,1063.0,994.0,21.0,165.0,22.0,50.0,...,None,None,None,None,None,None,None,None,None,None
4,Alabama,2019,5,2019-02-02,1095.0,1026.0,18.0,199.0,19.0,52.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Hawaii,2019,17,2019-04-27,221.0,211.0,NaN,49.0,NaN,NaN,...,None,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),None,None,None,None,None
996,Hawaii,2019,18,2019-05-04,209.0,190.0,NaN,37.0,NaN,NaN,...,None,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),None,None,None,None,None
997,Hawaii,2019,19,2019-05-11,246.0,232.0,NaN,52.0,NaN,NaN,...,None,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),None,None,None,None,None
998,Hawaii,2019,20,2019-05-18,233.0,213.0,NaN,53.0,NaN,NaN,...,None,Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),Suppressed (counts 1-9),None,None,None,None,None


In [33]:
# Confirm county data has been added
pd.read_sql_query('select * from county', con=engine)

,state,fips,county,county_seat,lat,lon
0,AL,1001,Autauga,Prattville,33,-87
1,AL,1003,Baldwin,Bay Minette,31,-88
2,AL,1005,Barbour,Clayton,32,-85
3,AL,1007,Bibb,Centreville,33,-87
4,AL,1009,Blount,Oneonta,34,-87
...,...,...,...,...,...,...
3138,WY,56037,Sweetwater,Green River,42,-109
3139,WY,56039,Teton,Jackson,44,-111
3140,WY,56041,Uinta,Evanston,41,-111
3141,WY,56043,Washakie,Worland,44,-108
